In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using LinearAlgebra
using KrylovKit
using QuadGK
using Plots
using LaTeXStrings

using ExactDiagonalizationTools
using MatrixProductStateTools

In [ ]:
xs = -1:0.01:1
sigmoid(x) = 1/(1. + exp(-x))
f2(x) = -0.03 .+ sigmoid.(-11.1 *abs(x) + 5.6)
#plot(xs, f2.(xs))

In [ ]:
maxdim = 40
gs = collect(0.8:0.1:1.2)
lxs = [2*n+1 for n=2:7]

data = Dict{Int, Vector{Float64}}()
envelopes = Vector{Float64}[]
rs = Vector{Float64}[]

for lx in lxs
    r = -(lx/2 .- collect(1:lx))./(lx/2)
    envelope = f2.(r) 
    envelope = envelope ./ maximum(envelope)
    #envelope = ones(lx)
    
    push!(rs, r)
    push!(envelopes, envelope)
    
    gdata = Float64[]
    for g in gs
        h = qitf_hamiltonian(lx, g, envelope=envelope) 
        bond = qitf_bondtensor(g)
        es, vs = eigsolve(h, 1, :SR)
        imps = ketstate2imps(vs[1], lx, 2, maxdim=maxdim)
        e_imps = measure_bond(imps, bond)

        e_exact, e_exact_err = quadgk(k -> -sqrt(1+g^2-2*g*cos(k))/pi, 0, pi)
        push!(gdata, e_imps-e_exact)
    end
    data[lx] = gdata
end

### PLOTTING ###
p1 = plot(xaxis=(L"$g$"), 
    yaxis=((1.e-5, 2.e-2),L"$E - E_{\infty}$", :log, [10. ^-i for i in 2:2:10]), 
    marker=(:o))
for lx in lxs
    plot!(p1, gs, data[lx], marker=(:c), label=latexstring("L = $lx"))
end
p2 = plot(yaxis=("envelope", (-0.1,1.1)), legend=false)
for i in eachindex(envelopes)
    plot!(p2, rs[i], envelopes[i])
end
p = plot(p1, p2, layout=@layout [a; b{.3h}])

In [ ]:
savefig(p, "sigmoid_slightshift.png")